In [ ]:
import pandas as pd
yr20_old = pd.read_csv('../codoxer/data/gcj-dataset-master/gcj2020.csv')
yr20_one = pd.read_csv('../codoxer/data/oneforone_2020.csv')
yr20 = pd.read_csv('../codoxer/data/gcj_data_select.csv')
yr20.head()

In [ ]:
yr20.columns

In [ ]:
#df = yr20.drop(columns= ['Unnamed: 0', 'file', 'year'])

In [ ]:
df = yr20.drop(columns = ['task', 'year'])

In [ ]:
#df = yr20.drop(columns= ['Unnamed: 0', 'full_path', 'round', 'solution', 'task', 'year'])

In [ ]:
import regex as re

In [ ]:
#df['lan'] = df['full_path'].apply(lambda x: x.split('.')[-1].lower())

In [ ]:
df = df[df.language.isin(['cpp', 'python', 'java'])]

In [ ]:
df['code_length'] = df['flines'].apply(lambda x: len(x))

In [ ]:
df['n_loops'] = df.flines.apply(lambda x: (x.count('for') + x.count('while')) / len(x) )

In [ ]:
df['n_imports'] = df.flines.apply(lambda x: (x.count('#include') + x.count('import')) / len(x) )

In [ ]:
df['n_contols'] = df.flines.apply(lambda x: (x.count('if') + x.count('else') + x.count('elif')) / len(x) )

In [ ]:
df['n_assigns'] = df.flines.apply(lambda x: (x.count('=') - x.count('==') - x.count('<=') - x.count('>=') - x.count('!=')) / len(x))

In [ ]:
df['n_asserts'] = df.flines.apply(lambda x: x.count('assert') / len(x))

In [ ]:
df['has_main'] = df.flines.apply(lambda x: min(x.count('main()'), 1))

In [ ]:
df['n_newlines'] = df.flines.apply(lambda x: x.count('\n') / len(x))

In [ ]:
df['n_whitespace'] = df.flines.apply(lambda x: x.count(' ') / len(x))

In [ ]:
def count_comments(language, code):
    if language == 'cpp':
        return code.count('//') + code.count('/*') + code.count('*/')
    if language == 'python':
        return code.count('#') + code.count('"""')
    if language == 'java':
        return code.count('//') + code.count('/*') + code.count('*/') + code.count('/**') + code.count('/**')

In [ ]:
df['n_comments'] = df.apply(lambda x: count_comments(x['language'], x['flines']) / len(x), axis = 1)

In [ ]:
df['n_camelCase'] = df.flines.apply(lambda x: sum(1 for m in re.finditer(r'[a-z][A-Z]', x)) / len(x))

In [ ]:
df['n_lines'] = df.flines.apply(lambda x: sum(1 for m in re.finditer(r'[^#\/]+', x)) / len(x))

In [ ]:
df['outputs'] = df.flines.apply(lambda x: (x.count('cout') + x.count('print(') + x.count('print ') + x.count('.print')) / len(x))

In [ ]:
#df['n_lines'] = df.flines.apply(lambda x: sum(1 for m in re.finditer(r'[\n\t].{1}[\n\t]', x)))


In [ ]:
#df['n_underscore'] = df.flines.apply(lambda x: x.count('_') / len(x))

In [ ]:
df['n_snakeCase'] = df.flines.apply(lambda x: sum(1 for m in re.finditer(r'[a-zA-Z]_[a-zA-Z]', x)) / len(x))

for idx in range(len(df.iloc[6].flines)):
    if df.iloc[6].flines[idx] != df.iloc[7].flines[idx]:
        print(str(idx) + ' ' + df.iloc[6].flines[idx] + df.iloc[7].flines[idx])

In [ ]:
#list(df[df.language == 'cpp'].username.value_counts().head(10).index)

In [ ]:
#df.username.value_counts()

In [ ]:
#data_iteration1 = df[df.language == 'cpp']
#data_iteration1 = df[df.username.isin(['dzhulgakov', 'vepifanov'])]
data_iteration1 = df[df.username.isin(list(df.username.value_counts().head(1154).index))]

In [ ]:
#print(data_iteration1.loc[441962].flines)

In [ ]:
#data_iteration1.sort_values(by = 'code_length', ascending=False)

In [ ]:
data_iteration1['user'] = data_iteration1.username.apply(lambda x: 1 if x == 'dzhulgakov' else 0)

In [ ]:
data_iteration1

In [ ]:
X = data_iteration1.drop(columns = ['flines', 'username', 'language'])


y = data_iteration1.username

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.multiclass import OneVsOneClassifier

In [ ]:
import seaborn as sns
sns.pairplot(X)

In [ ]:
#X.user.value_counts()

In [ ]:
#X = X.drop(columns = 'user')

In [ ]:
X.columns

In [ ]:
cl = ColumnTransformer([('scaler', RobustScaler(), ['code_length', 'n_loops', 'n_imports', 'n_contols', 'n_assigns',
       'n_asserts', 'n_newlines', 'n_whitespace', 'n_comments',
       'n_camelCase', 'n_lines', 'outputs', 'n_snakeCase'])])

In [ ]:
est = OneVsOneClassifier(LogisticRegression())

In [ ]:
pipe = make_pipeline(cl, est)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
from sklearn.model_selection import cross_val_score

cv_score = cross_val_score(pipe, X, y, cv = 10, n_jobs = -1, verbose = 1)

In [ ]:
cv_score.mean()

In [ ]:
pipe.fit(pd.DataFrame(X_train, columns=X.columns), y_train)

In [ ]:
pipe.score(X_test, y_test)

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

In [ ]:
pipe.steps[1][1].estimators_

In [ ]:
df[df.username.isin(['dzhulgakov', 'vepifanov'])].sort_values(by = 'username')

Cluster that shit

In [ ]:
from sklearn.cluster import k_means

In [ ]:
model = k_means(X, n_clusters = 10)

In [ ]:
pd.DataFrame(model[1], data_iteration1.username).sort_index()